In [ ]:
import os
import json
import pandas as pd
import calendar
import random
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import numpy as np
import math

In [ ]:
final_streaming_data_df = pd.read_pickle(os.path.join(os.getcwd(), 'final_streaming_data_df.pkl'))

In [ ]:
def color_distance(color1, color2):
    """Calculate the Euclidean distance between two colors."""
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(color1, color2)))

def is_color_similar(color, used_colors, threshold=100):
    """Check if the color is similar to any in the used colors."""
    return any(color_distance(color, used_color) < threshold for used_color in used_colors)

def generate_unique_color(used_colors, threshold=100):
    """Generate a unique color that isn't similar to any used color."""
    while True:
        color = tuple(random.randint(0, 255) for _ in range(3))
        if not is_color_similar(color, used_colors, threshold):
            return color

def calculate_luminance(color):
    """Calculate the luminance of a color."""
    return 0.299 * color[0] + 0.587 * color[1] + 0.114 * color[2]

def choose_text_color(background_color1, background_color2):
    """Choose black or white text based on the average luminance of the background colors."""
    avg_luminance = (calculate_luminance(background_color1) + calculate_luminance(background_color2)) / 2
    return (255, 255, 255) if avg_luminance < 128 else (0, 0, 0)

def add_noise_texture(size, color1, color2, noise_level, contrast_factor=1.0):
    # Create a linear gradient between color1 and color2
    gradient = np.linspace(0, 1, size[0])[:, None]
    gradient_image = np.zeros((size[0], size[1], 3), dtype=np.float32)

    # Create the gradient image
    for i in range(3):  # For R, G, B channels
        gradient_image[..., i] = gradient * (color2[i] - color1[i]) + color1[i]

    # Generate random noise
    noise = np.random.normal(0, noise_level, size).astype(np.float32)
    
    # Adjust the noise to increase contrast
    noise = noise * contrast_factor

    # Apply the noise to the gradient image
    noisy_image = np.clip(gradient_image + noise[..., None], 0, 255).astype(np.uint8)
    
    return Image.fromarray(noisy_image)

In [ ]:
def create_images_from_df(df, output_dir='covers'):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Image size (640x640 for Spotify cover)
    image_size = (640, 640)
    
    # Imaginary box size and margin
    box_size = (560, 560)
    border_margin = 40
    
    # Map month numbers to names
    month_mapping = {index: month for index, month in enumerate(calendar.month_name) if month}
    
    # Aggregate by year and month
    df_grouped = df.groupby(['year', 'month']).size().reset_index(name='count')
    
    # Dictionary to store colors for each year
    year_colors = {}

    # Track used colors to avoid similarity
    used_colors = []

    # Get the current working directory
    current_dir = os.getcwd()

    # Construct the relative font path
    font_path = os.path.join(current_dir, "fonts", "Paskowy.ttf")
    
    # Check if the font path is correct
    if not os.path.exists(font_path):
        raise FileNotFoundError(f"Font file not found at: {font_path}")

    # Iterate through each row in the grouped DataFrame
    for index, row in df_grouped.iterrows():
        # Convert month number to month name
        month_name = month_mapping[row['month']]
        # Abbreviate the month name for the text on the image
        month_name_abbrev = month_name[:3].upper()
        # Format the year as 'YY (e.g., '19)
        formatted_year = f"{str(row['year'])[-2:]}"
        text = f"{month_name_abbrev} {formatted_year}"
        
        # Assign a pair of random unique base colors for the year if not already assigned
        if row['year'] not in year_colors:
            color1 = generate_unique_color(used_colors)
            used_colors.append(color1)
            color2 = generate_unique_color(used_colors)
            used_colors.append(color2)
            year_colors[row['year']] = (color1, color2)
        
        # Use the assigned colors for the year
        color1, color2 = year_colors[row['year']]
        
        # Create a gradient noise background using the two colors
        background_img = add_noise_texture(image_size, color1, color2, noise_level=50, contrast_factor=1.5)
        
        # Determine the best text color (black or white) based on background colors
        text_color = choose_text_color(color1, color2)
        
        # Initialize ImageDraw
        d = ImageDraw.Draw(background_img)
        
        # Load the custom font
        try:
            font = ImageFont.truetype(font_path, 10)  # Start with a small font size
        except IOError:
            raise IOError(f"Cannot load font from path: {font_path}")
        
        # Adjust font size to fit within the imaginary box
        max_font_size = 200  # Starting point for maximum font size
        while True:
            font = ImageFont.truetype(font_path, max_font_size)
            text_width, text_height = d.textsize(text, font=font)
            if text_width <= box_size[0] and text_height <= box_size[1]:
                break
            max_font_size -= 1  # Reduce font size if it doesn't fit
            
        # Calculate the position to center the text within the image
        position = ((image_size[0] - text_width) // 2, (image_size[1] - text_height) // 2)
        
        # Add text to the image using the chosen text color
        d.text(position, text, font=font, fill=text_color)
        
        # Save the image with a properly ordered filename and ensure it meets the file size limit
        filename = f"{output_dir}/{row['year']}_{str(row['month']).zfill(2)}_{month_name_abbrev}.jpg"
        
        quality = 95
        while True:
            background_img.save(filename, format="JPEG", quality=quality)
            if os.path.getsize(filename) <= 256 * 1024 or quality <= 30:
                break
            quality -= 5

In [ ]:
create_images_from_df(final_streaming_data_df)